In [ ]:
%pylab inline
from numba import jit

In [ ]:
def fake_root_finder(func, a, b, arg):
    return (func(a, *arg) + func(b, *arg)) / 2

def root(t, u, v):
    func = t**3*(np.sin(t))**2+u*v
    return func

def find_root_loop(iter):
    for i in range(iter):
        r = np.random.random()
        fake_root_finder(root, -2.2, 5, arg=(r, 1))

In [ ]:
%time find_root_loop(1000000)

In [ ]:
@jit(nopython=True)
def fake_root_finder_jited(func, a, b, arg):
    return (func(a, *arg) + func(b, *arg)) / 2

@jit(nopython=True)
def root(t, u, v):
    func = t**3*(np.sin(t))**2+u*v
    return func

@jit(nopython=True)
def find_root_loop_jited(iter):
    for i in range(iter):
        r = np.random.random()
        fake_root_finder_jited(root, -2.2, 5, arg=(r, 1))

In [ ]:
%%time 
find_root_loop_jited(1000000)

In [ ]:
%time find_root_loop(1000000)
%time find_root_loop_jited(1000000)

In [ ]:
import numpy as np
import numba as nb
from numba import jit

def bisection(func, a, b, tol=1e-12, iter_nmax=100):
    fa = func(a)
    fb = func(b)
    if fa*fb >= 0: 
        raise ValueError("The root is not bracketed since f(a) and f(b) have the same sign.")
    if np.abs(fa)<np.abs(fb):
        temp = a
        a = b
        b = temp

    iter_n = 0
    while iter_n < iter_nmax:
        c = (a + b) / 2
        fc = func(c)
        if func(c) == 0 or (b-a)/2 < tol:
            # root found
            return c
        iter_n += 1

        if fc*fa:
            a = c
        else:
            b = c
    else:
        raise Exception("Root non-found. Exceeded max. number of iteration.")

bisection_jited = jit(nopython=True)(bisection)

In [ ]:
from time import time
def root(t):
    func = t**3*(np.sin(t))**2+1
    return func
root_jited = jit(nopython=True)(root)

loop_num = 100000
def loop_bisection(arg):
    result = 0
    for i in range(loop_num):
        rn = np.random.random()
        result += arg*rn*bisection(root, -2.2, 5)
@jit(nopython=True, parallel=True)
def loop_bisection_jited(arg):
    result = 0
    for i in nb.prange(loop_num):
        rn = np.random.random()
        result += arg*rn*bisection_jited(root_jited, -2.2, 5)

In [ ]:
%%time
loop_bisection(10)


In [ ]:
%%time
loop_bisection_jited(89)

In [ ]:
import numpy as np
import numba as nb
from numba import jit
from root_finder import bisection_arg, brentq

@jit(nopython=True, fastmath=True)
def root(t, u, v) -> float:
    func = t**3*(np.sin(t))**2 +u*v
    return func

def find_root_loop(iter):
    result = 0
    for i in nb.prange(iter):
        r = np.random.random()
        result += bisection_arg(root, -2.2, 5, (1.0, 1.0))
    return result/iter

find_root_loop_jited = jit(nopython=True, parallel=True, fastmath=True)(find_root_loop)
%time find_root_loop(300000)
%time find_root_loop_jited(300000)


In [ ]:
from numba import jit
import numba as nb
from tele_geo_jit import z1, m1_into_tele
z1_jit = jit(nopython=True, parallel=True, fastmath=True)(z1)
m1_into_tele_jit =  jit(nopython=True, fastmath=True)(m1_into_tele)

def z1_loop(iter):
    for i in nb.prange(iter):
        x = np.random.random()
        y = np.random.random()
        z1(x, y)
    return 0

@jit(nopython=True, parallel=True, fastmath=True)
def z1_loop_jit(iter):
    for i in nb.prange(iter):
        x = np.random.random()
        y = np.random.random()
        z1_jit(x, y)
    return 0

def m1_into_tele_loop(iter):
    for i in nb.prange(iter):
        x = np.random.random()
        y = np.random.random()
        z = np.random.random()
        m1_into_tele(x, y, z)
    return 0

@jit(nopython=True, parallel=True, fastmath=True)
def m1_into_tele_loop_jit(iter):
    for i in nb.prange(iter):
        x = np.random.random()
        y = np.random.random()
        z = np.random.random()
        m1_into_tele_jit(x, y, z)
    return 0


In [ ]:
%time z1_loop(20000)
%time m1_into_tele_loop(10000)

In [ ]:
%time z1_loop_jit(1)
%time m1_into_tele_loop_jit(1)

%time z1_loop_jit(20000)
%time m1_into_tele_loop_jit(10000)

In [13]:
import numpy as np
import numba as nb
from numba import jit

# @jit(nopython=True, parallel=True, fastmath=True)
def generate_theta_phi_array(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N):
    theta = np.linspace(theta_a, theta_b, theta_N)
    phi = np.linspace(phi_a, phi_b, phi_N)
    theta = theta.repeat(theta_N).reshape((theta_N, theta_N)).T.flatten()
    phi = phi.repeat(phi_N)
    return theta, phi
generate_theta_phi_array_jit = jit(nopython=True, parallel=True, fastmath=True)(generate_theta_phi_array)

theta_a, theta_b, theta_N = 0, 4, 4
phi_a, phi_b, phi_N = 0.1, 3.1, 4
theta, phi = generate_theta_phi_array(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N)
theta, phi = generate_theta_phi_array_jit(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N)


theta_orig = np.linspace(theta_a, theta_b, theta_N)
phi_orig = np.linspace(phi_a, phi_b, phi_N)
theta_orig, phi_orig = np.meshgrid(theta_orig, phi_orig)
theta_orig = np.ravel(theta_orig)
phi_orig = np.ravel(phi_orig)

assert theta_orig.all() == theta.all()
assert phi_orig.all() == phi.all()
print(theta_orig)
print(theta)
print(phi_orig)
print(phi)

theta_a, theta_b, theta_N = 0, 4, 500
phi_a, phi_b, phi_N = 0.1, 3.1, 500
%time theta, phi = generate_theta_phi_array(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N)
%time theta, phi = generate_theta_phi_array_jit(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N)

[0.         1.33333333 2.66666667 4.         0.         1.33333333
 2.66666667 4.         0.         1.33333333 2.66666667 4.
 0.         1.33333333 2.66666667 4.        ]
[0.         1.33333333 2.66666667 4.         0.         1.33333333
 2.66666667 4.         0.         1.33333333 2.66666667 4.
 0.         1.33333333 2.66666667 4.        ]
[0.1 0.1 0.1 0.1 1.1 1.1 1.1 1.1 2.1 2.1 2.1 2.1 3.1 3.1 3.1 3.1]
[0.1 0.1 0.1 0.1 1.1 1.1 1.1 1.1 2.1 2.1 2.1 2.1 3.1 3.1 3.1 3.1]
Wall time: 5 ms
Wall time: 4 ms


In [6]:
import numpy as np
import numba as nb
from numba import jit
from tele_geo_jit import *

@jit(nopython=True, fastmath=True)
def root_z2(t, x_0, y_0, z_0, alpha, beta, gamma):
    # Endpoint of ray:
    x = x_0 + alpha * t
    y = y_0 + beta * t
    z = z_0 + gamma * t

    # Convert to M2 r.f.
    xm2, ym2, zm2 = tele_into_m2(x, y, z)

    # Z of mirror in M2 r.f.
    z_m2 = z2(xm2, ym2)
    return zm2 - z_m2
t = 133.0
x_0, y_0, z_0, alpha, beta, gamma = 0, 0, 0, 1.2, 0.5, 0.1
result = root_z2(t, x_0, y_0, z_0, alpha, beta, gamma)

In [18]:
import numpy as np
diff = np.array([0.4e-11, 0.1e-11, 0.11e-11])
if diff.any() < 1e-11:
    print("OH Yes!")
else:
    print("nonono")

nonono


In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from time import time
import numpy as np
import plotly.graph_objects as go

from DEFAULTS import PARENT_PATH
import ap_field_jit as ap
import pan_mod_jit as pm
import tele_geo_jit as tg
import latrt_geo as lgeo

tele_geo_t = tg.initialize_telescope_geometry()

tg_th_1 =  tele_geo_t.th_1
tg_th2 = tele_geo_t.th2
tg_z_ap = tele_geo_t.z_ap
tg_th_fwhp = tele_geo_t.th_fwhp
tg_F_2 = tele_geo_t.F_2

tg_x_rotc = tele_geo_t.x_rotc
tg_y_rotc = tele_geo_t.y_rotc
tg_z_rotc = tele_geo_t.z_rotc

freq = 120.0 # frequency of signal source [GHz]
wavelength = (30.0 / freq) * 0.01
k = 2 * np.pi / wavelength

dist = np.inf # set it infinitly far away
P_source = np.array([0, -7.2, dist])  # position of the source[m] 

tg_el0 = np.arctan(-(P_source[1] + 7.2) / P_source[2]) # elevation of telescope based on position of source tower [rad]
tg_az0 = np.arctan(-P_source[0] / P_source[2]) # azimuth of telescope based on position of source tower [rad]

error = 0 # mean mirror surface error [um]
adj1 = np.random.randn(1092) * error
adj2 = np.random.randn(1092) * error
save = 0
pan_mod2 = pm.panel_model_from_adjuster_offsets(
    2, adj2, 1, save
)  # Panel Model on M2
pan_mod1 = pm.panel_model_from_adjuster_offsets(
    1, adj1, 1, save
)  # Panel Model on M1

N_scan = 64
theta = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, N_scan)
phi = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, N_scan)

focus = np.array([0,0,0]) #the focus point

In [2]:
P_rx = focus # Receiver feed position in focal plane
rxmirror_t = ap.ray_mirror_pts(P_rx, tg_th2, tg_F_2, theta, phi)
apfield_t = ap.aperature_fields_from_panel_model(pan_mod1, pan_mod2, \
                                        P_rx, tg_th_1, tg_th2, tg_z_ap, tg_th_fwhp, \
                                        tg_F_2, theta, phi, rxmirror_t
                                        )

In [3]:
######### draw the mirrors and focal plane #############

miiror_size = 6000.0/np.cos(np.arctan(1 / 2))
msf = miiror_size/2.0

# positions of mirror 1 surface
x_m1 = np.linspace(-msf,msf,10, endpoint=True) # [mm]
y_m1 = np.linspace(-msf,msf,10, endpoint=True) # [mm]
x_m1, y_m1 = np.meshgrid(x_m1, y_m1)
height_1 = lgeo.z1(x_m1, y_m1)
x_m1, y_m1, z_m1 = lgeo.m1_into_tele(x_m1, y_m1, height_1)

# positions of mirror 2 surface
x_m2 = np.linspace(-msf,msf,10, endpoint=True) # [mm]
y_m2 = np.linspace(-msf,msf,10, endpoint=True) # [mm]
x_m2, y_m2 = np.meshgrid(x_m2, y_m2)
height_2 = lgeo.z2(x_m2, y_m2)
x_m2, y_m2, z_m2 = lgeo.m2_into_tele(x_m2, y_m2, height_2)

# positions of focal plane 
x_fp = np.linspace(-2000,2000,100, endpoint=True) # [mm]
y_fp = np.linspace(-2000,2000,100, endpoint=True) # [mm]
x_fp, y_fp = np.meshgrid(x_fp, x_fp)
z_fp = lgeo.z_focal(x_fp,y_fp)
r = np.sqrt(x_fp**2 + y_fp**2)
z_fp = np.where(r<2000,z_fp,np.nan)
x_fp,y_fp,z_fp = lgeo.foc_into_tele(x_fp,y_fp,z_fp,tg_el0, tg_az0)

plot_m1 = go.Surface(
                        x = x_m1,
                        y = y_m1,
                        z = z_m1,
                        surfacecolor=height_1, 
                        colorscale='Teal',
                        showscale= False, 
                        name = 'Mirror 1',
                        text="Mirror 1", 
                        showlegend = True, 
                        hoverinfo="name"
                    )

plot_m2 = go.Surface(
                        x = x_m2,
                        y = y_m2,
                        z = z_m2,
                        surfacecolor=-height_2,
                        colorscale='Teal',
                        showscale= False, 
                        name = 'Mirror 2',
                        showlegend = True, 
                        hoverinfo="name"
                    )

plot_fp = go.Surface(
                        x = x_fp,
                        y = y_fp,
                        z = z_fp,
                        surfacecolor=z_fp*0 + 0.6,
                        colorscale='Cividis',
                        showscale= False, 
                        opacity=0.4,
                        name = 'Focal Plane',
                        showlegend = True, 
                        hoverinfo="name"
                    )

plot_setup = [plot_m1, plot_m2, plot_fp]


######### draw the rays obtained from the simulation #############
apfield_t = apfield_t
fp = np.where(apfield_t[15,:]!=0) #Ignore rays that missed the focal plane
amp_min = np.min(apfield_t[16,:][fp])
amp_max = np.max(apfield_t[16,:][fp])
x_foc, y_foc, z_foc = P_rx

plot_rays = []
x_a_list, y_a_list, z_a_list = [], [], []
amp_list = []
for ii in np.unique(np.random.randint(low=0, high=len(apfield_t[15]), size=1000)):
    if apfield_t[15][ii] !=0 :
        x_m1, y_m1, z_m1 = apfield_t[3][ii], apfield_t[4][ii], apfield_t[5][ii]
        x_m2, y_m2, z_m2 = apfield_t[0][ii], apfield_t[1][ii], apfield_t[2][ii]
        x_a, y_a, z_a = apfield_t[9][ii], apfield_t[10][ii], apfield_t[11][ii]


        xline = [ x_a, x_m1, x_m2, x_foc]
        yline = [ y_a, y_m1, y_m2, y_foc]
        zline = [ z_a, z_m1, z_m2, z_foc]
    
        amp = apfield_t[16,:][ii]

        my_color = ('rgba('+str(np.random.randint(50, high = 200))+','+
                    str(np.random.randint(50, high = 200))+','+
                    str(np.random.randint(50, high = 200)))

        ray_line =go.Scatter3d(
                            x=xline, 
                            y=yline, 
                            z=zline,
                            mode='lines',
                            line=dict(
                                    color=[amp,amp,amp,amp],
                                    cmax=amp_max, 
                                    cmin=amp_min,
                                    width=5, 
                                    colorscale='Purpor', 
                                    ), 
                            opacity=0.4,
                            showlegend = False,
                            hoverinfo='none', 
                            )

        plot_rays.append(ray_line)
        x_a_list.append(x_a)
        y_a_list.append(y_a)
        z_a_list.append(z_a)
        amp_list.append(amp)

plot_ap = [go.Scatter3d(
                        x = x_a_list, y = y_a_list, z = z_a_list, 
                        mode = 'markers', 
                        marker=dict(symbol="circle", size=5, colorscale='Purpor', color=amp_list), 
                        opacity=0.4,
                        showlegend = True,
                        hoverinfo='name', 
                        name="rays"
                      )
          ]

#################### show the plot #########################
plot_layout = go.Layout(title='LAT', autosize=False,
                  width=1000, height=600,
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene = dict(
                                xaxis=dict(visible=False),
                                yaxis=dict(visible=False),
                                zaxis=dict(visible=False),
                                aspectmode='data'),
                )
fig = go.Figure(
                data=plot_setup + plot_rays + plot_ap, 
                layout=plot_layout
                )
fig.show()

In [25]:

# save_parameter()
# with open(f"{path}/{filename}.pys", "rb") as file:
#     read = pickle.load(file)
# print(read)

In [4]:

import numba as nb
from numba import jit
import pickle

path = f"{PARENT_PATH}/data/tung/JIT_first_test"  # path where you save far-field simulations.
if not os.path.exists(path):
    os.makedirs(path)

fp_x_N = 200
fp_z_N = 200
range_factor = 64
fp_x = np.linspace(focus[0]-range_factor*wavelength, focus[0]+range_factor*wavelength, 200) # unit: [m]
fp_z = np.linspace(focus[2]-range_factor*wavelength, focus[2]+range_factor*wavelength, 200)  # unit: [m]

N_scan = 64
theta = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, N_scan)
phi = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, N_scan)

def save_parameter():
    for ii in range(10):
        rn = np.random.random()
        save_parameters = dict(
                            P_source=P_source, 
                            surface_error=error, 
                            fp_x_N=fp_x_N, 
                            fp_z_N=fp_z_N, 
                            wavelength=wavelength, 
                            randnum=rn
                        )
        filename = "parameters_%i_%f"%(ii, rn)
        with open(f"{path}/{filename}.pys", "wb") as file:
            pickle.dump(save_parameters, file)

sim_size = fp_x_N*fp_z_N
savearray = np.zeros((sim_size, 17, N_scan*N_scan))

@jit(nopython=True, parallel=True, fastmath=True)
def simulation(fp_x_N, fp_z_N):
    savearray = np.zeros((sim_size, 17, N_scan*N_scan))
    for ii in nb.prange(fp_x_N):
        for jj in nb.prange(fp_z_N):
            P_rx = np.array([fp_x[ii], focus[2], fp_z[ii]])
            rxmirror_t = ap.ray_mirror_pts(P_rx, tg_th2, tg_F_2, theta, phi)
            apfield_t = ap.aperature_fields_from_panel_model(pan_mod1, pan_mod2, \
                                                    P_rx, tg_th_1, tg_th2, tg_z_ap, tg_th_fwhp, \
                                                    tg_F_2, theta, phi, rxmirror_t
                                                    )
            savearray[ii*fp_x_N+jj,:,:] = apfield_t
            # print("Progress:", "(",  (ii*fp_x_N+jj+1) * 100.0 / sim_size, '% already done)')
simulation(fp_x_N, fp_z_N)
filename = "apfield_sim"
with open(f"{path}/{filename}.pys", "wb") as file:
    pickle.dump(savearray, file)
